# PowerModelsDistribution.jl Transformer Modelling Tutorial

`PowerModelsDistribution.jl` allows modelling power transformers with any number of windings and number of phases taking into account the loss models for each winding. While this is a powerful feature, it can be complex to model and confusing for starting users. Hence, this tutorial is to provide a simple introduction to power transformer models in `PowerModelsDistribution.jl`. For the full details on how the transformer model was implemented please refer to S. Claeys, G. Deconinck and F. Geth, “Decomposition of n-winding transformers for unbalanced optimal power flow,” IET Generation, Transmission & Distribution, vol. 14, no. 24, pp. 5961-5969, 2020, DOI: 10.1049/iet-gtd.2020.0776.


## Transformer Modelling Theory

Starting with the simplest case of a two-winding, single-phase transformer, which will later serve as the fundamental building block for $n$-winding, $n$-phase transformers.

Without delving deeply into the electromagnetic theory of transformer operation, the functioning of a transformer can be simplified as follows: a primary coil (coil 1) and a secondary coil (coil 2) are wound on the same magnetic core. When a voltage $U_1$ is applied to the primary coil, it generates a magnetic flux $\Phi_1$. Part of this flux links the primary coil with itself (not going through the core to the secondary coil), denoted as $\Phi_{11}$, while another part, denoted as $\Phi_{m}$, links the secondary coil. The magnetic flux $\Phi_{m}$ induces a voltage $U_2$ in the secondary coil, which can drive a current $I_2$ when a load is connected to the secondary coil.



<img src="https://github.com/MohamedNumair/PMD-Transformer-Model-Tutorial/blob/main/transformer_fields.svg" width="1000">

Taking into account that each of the coils has a resistance $R_1$ and $R_2$ and modelling the leakage flux $\Phi_{11}$ and $\Phi_{22}$



In [914]:
using Pkg
Pkg.activate(".")
Pkg.develop("PowerModelsDistribution")


  Activating project at `c:\Users\mnumair\OneDrive - KU Leuven\PhD Agenda\2025\12-25 December\PMD Transformer Model Tutorial`
   Resolving package versions...
     Project No packages added to or removed from `C:\Users\mnumair\OneDrive - KU Leuven\PhD Agenda\2025\12-25 December\PMD Transformer Model Tutorial\Project.toml`
    Manifest No packages added to or removed from `C:\Users\mnumair\OneDrive - KU Leuven\PhD Agenda\2025\12-25 December\PMD Transformer Model Tutorial\Manifest.toml`


In [915]:
Pkg.status()

Status `C:\Users\mnumair\OneDrive - KU Leuven\PhD Agenda\2025\12-25 December\PMD Transformer Model Tutorial\Project.toml`
  [b6b21f68] Ipopt v1.13.0
  [d7431456] PowerModelsDistribution v0.16.0 `C:\Users\mnumair\.julia\dev\PowerModelsDistribution`


In [916]:
using PowerModelsDistribution

In [ ]:
function show_transformer_math_components(math)


end


In [917]:
pmd_file_path = joinpath(pathof(PowerModelsDistribution), "..\\test\\data")

"C:\\Users\\mnumair\\.julia\\dev\\PowerModelsDistribution\\src\\PowerModelsDistribution.jl\\..\\test\\data"

## Single-Phase Two-Winding Wye-Wye Transformer

In [919]:
eng_1ph_2w_yy = parse_file("trans_1ph_2w_yy.dss", transformations=[transform_loops!] )

[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 1 in 'trans_1ph_2w_yy.dss'
[ PowerModelsDistribution | Info ] : Command 'batchedit' on line 32 in 'trans_1ph_2w_yy.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'calcvoltagebases' on line 39 in 'trans_1ph_2w_yy.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'solve' on line 42 in 'trans_1ph_2w_yy.dss' is not supported, skipping.
┌ PowerModelsDistribution | Warning ] : basefreq=60.0 on line.line2 does not match circuit basefreq=50.0
└ @ PowerModelsDistribution C:\Users\mnumair\.julia\dev\PowerModelsDistribution\src\data_model\transformations\dss2eng.jl:422
┌ PowerModelsDistribution | Warning ] : basefreq=60.0 on line.line1 does not match circuit basefreq=50.0
└ @ PowerModelsDistribution C:\Users\mnumair\.julia\dev\PowerModelsDistribution\src\data_model\transformations\dss2eng.jl:422


Dict{String, Any} with 11 entries:
  "xfmrcode"       => Dict{String, Any}("tx"=>Dict{String, Any}("sm_ub"=>750.0,…
  "conductor_ids"  => [1, 2, 3, 4]
  "bus"            => Dict{String, Any}("sourcebus"=>Dict{String, Any}("rg"=>[0…
  "name"           => "ut_trans"
  "settings"       => Dict{String, Any}("sbase_default"=>1000.0, "vbases_defaul…
  "files"          => ["trans_1ph_2w_yy.dss"]
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("source_id"…
  "line"           => Dict{String, Any}("line2"=>Dict{String, Any}("cm_ub"=>[60…
  "data_model"     => ENGINEERING
  "transformer"    => Dict{String, Any}("tx1"=>Dict{String, Any}("polarity"=>[1…
  "load"           => Dict{String, Any}("load1"=>Dict{String, Any}("source_id"=…

In [920]:
eng_1ph_2w_yy["settings"]

Dict{String, Any} with 5 entries:
  "sbase_default"        => 1000.0
  "vbases_default"       => Dict{String, Real}("sourcebus"=>6.35085)
  "voltage_scale_factor" => 1000.0
  "power_scale_factor"   => 1000.0
  "base_frequency"       => 50.0

In [921]:
eng_1ph_2w_yy["transformer"]["tx1"]

Dict{String, Any} with 18 entries:
  "polarity"      => [1, 1]
  "sm_nom"        => [500.0, 500.0]
  "connections"   => [[1, 4], [1, 4]]
  "tm_lb"         => [[0.9], [0.9]]
  "tm_set"        => [[1.05], [0.95]]
  "tm_step"       => [[0.03125], [0.03125]]
  "bus"           => ["1", "2"]
  "configuration" => ConnConfig[WYE, WYE]
  "status"        => ENABLED
  "noloadloss"    => 0.05
  "xsc"           => [0.05]
  "cmag"          => 0.11
  "sm_ub"         => 750.0
  "source_id"     => "transformer.tx1"
  "rw"            => [0.01, 0.02]
  "tm_fix"        => Vector{Bool}[[1], [1]]
  "vm_nom"        => [11.0, 0.4]
  "tm_ub"         => [[1.1], [1.1]]

In [922]:
11.0/0.4

27.5

In [923]:
eng_1ph_2w_yy["transformer"]["tx1"]["vm_nom"][1] /eng_1ph_2w_yy["transformer"]["tx1"]["vm_nom"][2]

27.5

In [924]:
math_1ph_2w_yy = transform_data_model(eng_1ph_2w_yy)

Dict{String, Any} with 18 entries:
  "is_kron_reduced" => true
  "conductor_ids"   => [1, 2, 3]
  "time_elapsed"    => 1.0
  "bus"             => Dict{String, Any}("8"=>Dict{String, Any}("vm_pair_lb"=>T…
  "name"            => "ut_trans"
  "map"             => Dict{String, Any}[Dict("unmap_function"=>"_map_math2eng_…
  "settings"        => Dict{String, Any}("sbase_default"=>1000.0, "vbases_defau…
  "gen"             => Dict{String, Any}("1"=>Dict{String, Any}("pg"=>[0.0, 0.0…
  "branch"          => Dict{String, Any}("4"=>Dict{String, Any}("br_r"=>[0.0;;]…
  "storage"         => Dict{String, Any}()
  "switch"          => Dict{String, Any}()
  "is_projected"    => true
  "per_unit"        => true
  "data_model"      => MATHEMATICAL
  "shunt"           => Dict{String, Any}()
  "transformer"     => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>"_…
  "bus_lookup"      => Dict{Any, Int64}("sourcebus"=>1, "1"=>2, "2"=>3, "3"=>4)
  "load"            => Dict{String, Any}("1"=>Dict{Strin

In [925]:
math_1ph_2w_yy["map"][2]

Dict{String, Any} with 3 entries:
  "to"             => "bus.1"
  "from"           => "sourcebus"
  "unmap_function" => "_map_math2eng_bus!"

In [926]:
transformer_index_byname = findall(x->haskey(x, "from") && x["from"] == "tx1", math_1ph_2w_yy["map"])
math_1ph_2w_yy["map"][transformer_index_byname...]["to"]


9-element Vector{String}:
 "bus.5"
 "bus.6"
 "bus.7"
 "bus.8"
 "branch.3"
 "branch.4"
 "branch.5"
 "transformer.1"
 "transformer.2"

In [927]:
for element in math_1ph_2w_yy["map"][transformer_index_byname...]["to"]

    element_type = split(element, ".")[1]
    element_idx = split(element, ".")[2]
    display(element_type)
    display(element_idx)    
    display(math_1ph_2w_yy[element_type][element_idx])

end


"bus"

"5"

Dict{String, Any} with 13 entries:
  "vm_pair_lb" => Tuple{Any, Any, Real}[]
  "source_id"  => "transformer.tx1"
  "grounded"   => Bool[0]
  "vmin"       => [0.0]
  "vm_pair_ub" => Tuple{Any, Any, Real}[]
  "bus_i"      => 5
  "index"      => 5
  "name"       => "_virtual_bus.transformer.tx1_4"
  "bus_type"   => 1
  "terminals"  => [1]
  "vmax"       => [Inf]
  "base_kv"    => 1.0
  "vbase"      => 0.57735

"bus"

"6"

...

"1"

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.tx1.1"
  "t_connections" => [1]
  "f_bus"         => 2
  "polarity"      => 1
  "sm_ub"         => 0.75
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1]
  "tm_lb"         => [0.9]
  "tm_set"        => [1.05]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125]
  "t_bus"         => 8
  "f_connections" => [1]
  "configuration" => WYE
  "index"         => 1
  "name"          => "_virtual_transformer.tx1.1"
  "tm_nom"        => 1.0
  "status"        => 1
  "tm_ub"         => [1.1]
  "f_vbase"       => 6.35085

"transformer"

"2"

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.tx1.2"
  "t_connections" => [1]
  "f_bus"         => 3
  "polarity"      => 1
  "sm_ub"         => 0.75
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1]
  "tm_lb"         => [0.9]
  "tm_set"        => [0.95]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125]
  "t_bus"         => 6
  "f_connections" => [1]
  "configuration" => WYE
  "index"         => 2
  "name"          => "_virtual_transformer.tx1.2"
  "tm_nom"        => 1.0
  "status"        => 1
  "tm_ub"         => [1.1]
  "f_vbase"       => 0.23094

In [928]:
math_1ph_2w_yy["transformer"]

Dict{String, Any} with 2 entries:
  "1" => Dict{String, Any}("source_id"=>"_virtual_transformer.transformer.tx1.1…
  "2" => Dict{String, Any}("source_id"=>"_virtual_transformer.transformer.tx1.2…

In [929]:
math_1ph_2w_yy["transformer"]["1"]

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.tx1.1"
  "t_connections" => [1]
  "f_bus"         => 2
  "polarity"      => 1
  "sm_ub"         => 0.75
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1]
  "tm_lb"         => [0.9]
  "tm_set"        => [1.05]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125]
  "t_bus"         => 8
  "f_connections" => [1]
  "configuration" => WYE
  "index"         => 1
  "name"          => "_virtual_transformer.tx1.1"
  "tm_nom"        => 1.0
  "status"        => 1
  "tm_ub"         => [1.1]
  "f_vbase"       => 6.35085

In [930]:
math_1ph_2w_yy["transformer"]["2"]

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.tx1.2"
  "t_connections" => [1]
  "f_bus"         => 3
  "polarity"      => 1
  "sm_ub"         => 0.75
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1]
  "tm_lb"         => [0.9]
  "tm_set"        => [0.95]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125]
  "t_bus"         => 6
  "f_connections" => [1]
  "configuration" => WYE
  "index"         => 2
  "name"          => "_virtual_transformer.tx1.2"
  "tm_nom"        => 1.0
  "status"        => 1
  "tm_ub"         => [1.1]
  "f_vbase"       => 0.23094

In [931]:
(math_1ph_2w_yy["transformer"]["1"]["f_vbase"]/math_1ph_2w_yy["transformer"]["1"]["t_vbase"]) * (math_1ph_2w_yy["transformer"]["2"]["t_vbase"]/math_1ph_2w_yy["transformer"]["2"]["f_vbase"])

27.499999999999996

To get the ratio for the whole transformer - you divide the `f_vbase` of the first ideal transformer by the `f_vbase` of the second ideal transformer. (this will match the ratio of the real transformers)

In [932]:
math_1ph_2w_yy["transformer"]["1"]["f_vbase"]/math_1ph_2w_yy["transformer"]["2"]["f_vbase"]

27.499999999999993

Note this matches whether the LL voltages ration or the LN voltages ratio

In [933]:
# LL voltages ration
11/4

2.75

In [934]:
# LN voltages ration
6.35/2.3094

2.749631939031783

Summary Notes on 1-Ph Wye-Wye


We are looking at the LN to LN voltage ratio being 6.35/2.3094 (which was defined in OpenDSS LL voltages 11/4)


In `PMD.jl` this transformer is split into two ideal transformer one step down 6.35 (`f_vbase`) to 0.57735 (`t_vbase`) and another step up 0.57735 (`t_vbase`) to 2.3094 (`f_vbase`)
NOTE: the arbitrary virtual loss model voltage (0.57735) is alway on the to `t_` side of the transformer meaning that the second transformer is flipped kind of 

## Three-Phase Two-Winding Wye-Wye Transformer

In [935]:
eng_3ph_2w_yy = parse_file(joinpath(dirname(pathof(PowerModelsDistribution)), "..\\test\\data\\opendss\\ut_trans_2w_yy.dss"), transformations=[transform_loops!] )

[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 1 in 'ut_trans_2w_yy.dss'
[ PowerModelsDistribution | Info ] : Command 'calcvoltagebases' on line 35 in 'ut_trans_2w_yy.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'solve' on line 38 in 'ut_trans_2w_yy.dss' is not supported, skipping.


Dict{String, Any} with 10 entries:
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("source_id"…
  "name"           => "ut_trans"
  "line"           => Dict{String, Any}("line2"=>Dict{String, Any}("cm_ub"=>[60…
  "conductor_ids"  => [1, 2, 3, 4]
  "settings"       => Dict{String, Any}("sbase_default"=>1.0, "vbases_default"=…
  "files"          => ["C:\\Users\\mnumair\\.julia\\dev\\PowerModelsDistributio…
  "transformer"    => Dict{String, Any}("tx1"=>Dict{String, Any}("polarity"=>[1…
  "load"           => Dict{String, Any}("load3"=>Dict{String, Any}("source_id"=…
  "bus"            => Dict{String, Any}("1"=>Dict{String, Any}("rg"=>[0.0], "gr…
  "data_model"     => ENGINEERING

In [937]:
eng_3ph_2w_yy

Dict{String, Any} with 10 entries:
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("source_id"…
  "name"           => "ut_trans"
  "line"           => Dict{String, Any}("line2"=>Dict{String, Any}("cm_ub"=>[60…
  "conductor_ids"  => [1, 2, 3, 4]
  "settings"       => Dict{String, Any}("sbase_default"=>1.0, "vbases_default"=…
  "files"          => ["C:\\Users\\mnumair\\.julia\\dev\\PowerModelsDistributio…
  "transformer"    => Dict{String, Any}("tx1"=>Dict{String, Any}("polarity"=>[1…
  "load"           => Dict{String, Any}("load3"=>Dict{String, Any}("source_id"=…
  "bus"            => Dict{String, Any}("1"=>Dict{String, Any}("rg"=>[0.0], "gr…
  "data_model"     => ENGINEERING

In [938]:
eng_3ph_2w_yy["transformer"]["tx1"]

Dict{String, Any} with 19 entries:
  "polarity"      => [1, 1]
  "sm_nom"        => [500.0, 500.0]
  "tm_lb"         => [[0.9, 0.9, 0.9], [0.9, 0.9, 0.9]]
  "connections"   => [[1, 2, 3, 4], [1, 2, 3, 4]]
  "tm_set"        => [[1.02, 1.02, 1.02], [0.97, 0.97, 0.97]]
  "tm_step"       => [[0.03125, 0.03125, 0.03125], [0.03125, 0.03125, 0.03125]]
  "bus"           => ["1", "2"]
  "configuration" => ConnConfig[WYE, WYE]
  "name"          => "tx1"
  "status"        => ENABLED
  "noloadloss"    => 0.05
  "cmag"          => 0.11
  "xsc"           => [0.05]
  "source_id"     => "transformer.tx1"
  "sm_ub"         => 750.0
  "rw"            => [0.01, 0.02]
  "tm_fix"        => Vector{Bool}[[1, 1, 1], [1, 1, 1]]
  "vm_nom"        => [11.0, 4.0]
  "tm_ub"         => [[1.1, 1.1, 1.1], [1.1, 1.1, 1.1]]

In [939]:
eng_3ph_2w_yy["transformer"]["tx1"]["vm_nom"][1] /eng_3ph_2w_yy["transformer"]["tx1"]["vm_nom"][2]

2.75

In [940]:
math_3ph_2w_yy = transform_data_model(eng_3ph_2w_yy)

Dict{String, Any} with 18 entries:
  "is_kron_reduced" => true
  "conductor_ids"   => [1, 2, 3]
  "time_elapsed"    => 1.0
  "bus"             => Dict{String, Any}("8"=>Dict{String, Any}("vm_pair_lb"=>T…
  "name"            => "ut_trans"
  "map"             => Dict{String, Any}[Dict("unmap_function"=>"_map_math2eng_…
  "settings"        => Dict{String, Any}("sbase_default"=>1.0, "vbases_default"…
  "gen"             => Dict{String, Any}("1"=>Dict{String, Any}("pg"=>[0.0, 0.0…
  "branch"          => Dict{String, Any}("4"=>Dict{String, Any}("br_r"=>[0.0 0.…
  "storage"         => Dict{String, Any}()
  "switch"          => Dict{String, Any}()
  "is_projected"    => true
  "per_unit"        => true
  "data_model"      => MATHEMATICAL
  "shunt"           => Dict{String, Any}()
  "transformer"     => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>"_…
  "bus_lookup"      => Dict{Any, Int64}("1"=>1, "sourcebus"=>2, "2"=>3, "3"=>4)
  "load"            => Dict{String, Any}("4"=>Dict{Strin

In [941]:
transformer_index_byname = findall(x->haskey(x, "from") && x["from"] == "tx1", math_3ph_2w_yy["map"])
math_3ph_2w_yy["map"][transformer_index_byname...]["to"]


9-element Vector{String}:
 "bus.5"
 "bus.6"
 "bus.7"
 "bus.8"
 "branch.3"
 "branch.4"
 "branch.5"
 "transformer.1"
 "transformer.2"

In [942]:
for element in math_3ph_2w_yy["map"][transformer_index_byname...]["to"]

    element_type = split(element, ".")[1]
    element_idx = split(element, ".")[2]
    display(element_type)
    display(element_idx)    
    display(math_3ph_2w_yy[element_type][element_idx])

end


"bus"

"5"

Dict{String, Any} with 13 entries:
  "vm_pair_lb" => Tuple{Any, Any, Real}[]
  "source_id"  => "transformer.tx1"
  "grounded"   => Bool[0, 0, 0]
  "vmin"       => [0.0, 0.0, 0.0]
  "vm_pair_ub" => Tuple{Any, Any, Real}[]
  "bus_i"      => 5
  "index"      => 5
  "name"       => "_virtual_bus.transformer.tx1_4"
  "bus_type"   => 1
  "terminals"  => [1, 2, 3]
  "vmax"       => [Inf, Inf, Inf]
  "base_kv"    => 1.0
  "vbase"      => 0.57735

"bus"

"6"

...

"1"

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.tx1.1"
  "t_connections" => [1, 2, 3]
  "f_bus"         => 1
  "polarity"      => 1
  "sm_ub"         => 750.0
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1, 1, 1]
  "tm_lb"         => [0.9, 0.9, 0.9]
  "tm_set"        => [1.02, 1.02, 1.02]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125, 0.03125, 0.03125]
  "t_bus"         => 8
  "f_connections" => [1, 2, 3]
  "configuration" => WYE
  "index"         => 1
  "name"          => "_virtual_transformer.tx1.1"
  "tm_nom"        => 1.0
  "status"        => 1
  "tm_ub"         => [1.1, 1.1, 1.1]
  "f_vbase"       => 6.35085

"transformer"

"2"

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.tx1.2"
  "t_connections" => [1, 2, 3]
  "f_bus"         => 3
  "polarity"      => 1
  "sm_ub"         => 750.0
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1, 1, 1]
  "tm_lb"         => [0.9, 0.9, 0.9]
  "tm_set"        => [0.97, 0.97, 0.97]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125, 0.03125, 0.03125]
  "t_bus"         => 6
  "f_connections" => [1, 2, 3]
  "configuration" => WYE
  "index"         => 2
  "name"          => "_virtual_transformer.tx1.2"
  "tm_nom"        => 1.0
  "status"        => 1
  "tm_ub"         => [1.1, 1.1, 1.1]
  "f_vbase"       => 2.3094

In [943]:
math_3ph_2w_yy["transformer"]

Dict{String, Any} with 2 entries:
  "1" => Dict{String, Any}("source_id"=>"_virtual_transformer.transformer.tx1.1…
  "2" => Dict{String, Any}("source_id"=>"_virtual_transformer.transformer.tx1.2…

In [944]:
math_3ph_2w_yy["transformer"]["1"]

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.tx1.1"
  "t_connections" => [1, 2, 3]
  "f_bus"         => 1
  "polarity"      => 1
  "sm_ub"         => 750.0
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1, 1, 1]
  "tm_lb"         => [0.9, 0.9, 0.9]
  "tm_set"        => [1.02, 1.02, 1.02]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125, 0.03125, 0.03125]
  "t_bus"         => 8
  "f_connections" => [1, 2, 3]
  "configuration" => WYE
  "index"         => 1
  "name"          => "_virtual_transformer.tx1.1"
  "tm_nom"        => 1.0
  "status"        => 1
  "tm_ub"         => [1.1, 1.1, 1.1]
  "f_vbase"       => 6.35085

In [945]:
math_3ph_2w_yy["transformer"]["2"]

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.tx1.2"
  "t_connections" => [1, 2, 3]
  "f_bus"         => 3
  "polarity"      => 1
  "sm_ub"         => 750.0
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1, 1, 1]
  "tm_lb"         => [0.9, 0.9, 0.9]
  "tm_set"        => [0.97, 0.97, 0.97]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125, 0.03125, 0.03125]
  "t_bus"         => 6
  "f_connections" => [1, 2, 3]
  "configuration" => WYE
  "index"         => 2
  "name"          => "_virtual_transformer.tx1.2"
  "tm_nom"        => 1.0
  "status"        => 1
  "tm_ub"         => [1.1, 1.1, 1.1]
  "f_vbase"       => 2.3094

In [946]:
(math_3ph_2w_yy["transformer"]["1"]["f_vbase"]/math_3ph_2w_yy["transformer"]["1"]["t_vbase"]) * (math_3ph_2w_yy["transformer"]["2"]["t_vbase"]/math_3ph_2w_yy["transformer"]["2"]["f_vbase"])

2.7499999999999996

To get the ratio for the whole transformer - you divide the `f_vbase` of the first ideal transformer by the `f_vbase` of the second ideal transformer. (this will match the ratio of the real transformers)

In [947]:
math_3ph_2w_yy["transformer"]["1"]["f_vbase"]/math_3ph_2w_yy["transformer"]["2"]["f_vbase"]

2.7499999999999996

Note this matches whether the LL voltages ration or the LN voltages ratio

In [948]:
# LL voltages ration
11/4

2.75

In [949]:
# LN voltages ration
6.35/2.3094

2.749631939031783

SO Basicallly just the same as the single-phase wye-wye just the difference being that now the loss model (virtual branches) have 3x3 impedance matrices (not 4x4 though!)

## Three-Phase Explicit Neutral Two-Winding Wye-Wye Transformer

In [950]:
eng_3ph_en_2w_yy = parse_file(joinpath(dirname(pathof(PowerModelsDistribution)), "..\\test\\data\\en_validation_case_data\\test_trans_yy.dss"), transformations=[transform_loops!] )

[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 3 in 'test_trans_yy.dss'
[ PowerModelsDistribution | Info ] : Command 'calcvoltagebases' on line 36 in 'test_trans_yy.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'solve' on line 38 in 'test_trans_yy.dss' is not supported, skipping.
┌ PowerModelsDistribution | Warning ] : reactors as constant impedance elements is not yet supported, treating reactor.grounding like line
└ @ PowerModelsDistribution C:\Users\mnumair\.julia\dev\PowerModelsDistribution\src\data_model\transformations\dss2eng.jl:263


Dict{String, Any} with 12 entries:
  "conductor_ids"  => [1, 2, 3, 4, 5]
  "bus"            => Dict{String, Any}("x1"=>Dict{String, Any}("rg"=>[0.0], "g…
  "name"           => "test"
  "settings"       => Dict{String, Any}("sbase_default"=>200.0, "vbases_default…
  "files"          => ["C:\\Users\\mnumair\\.julia\\dev\\PowerModelsDistributio…
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("source_id"…
  "line"           => Dict{String, Any}("line2"=>Dict{String, Any}("length"=>50…
  "data_model"     => ENGINEERING
  "transformer"    => Dict{String, Any}("transformer1"=>Dict{String, Any}("pola…
  "shunt"          => Dict{String, Any}("line_loop.grounding"=>Dict{String, Any…
  "load"           => Dict{String, Any}("load1"=>Dict{String, Any}("source_id"=…
  "linecode"       => Dict{String, Any}("zabcn"=>Dict{String, Any}("b_fr"=>[0.0…

In [951]:
11.0/0.4

27.5

In [952]:
eng_3ph_en_2w_yy

Dict{String, Any} with 12 entries:
  "conductor_ids"  => [1, 2, 3, 4, 5]
  "bus"            => Dict{String, Any}("x1"=>Dict{String, Any}("rg"=>[0.0], "g…
  "name"           => "test"
  "settings"       => Dict{String, Any}("sbase_default"=>200.0, "vbases_default…
  "files"          => ["C:\\Users\\mnumair\\.julia\\dev\\PowerModelsDistributio…
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("source_id"…
  "line"           => Dict{String, Any}("line2"=>Dict{String, Any}("length"=>50…
  "data_model"     => ENGINEERING
  "transformer"    => Dict{String, Any}("transformer1"=>Dict{String, Any}("pola…
  "shunt"          => Dict{String, Any}("line_loop.grounding"=>Dict{String, Any…
  "load"           => Dict{String, Any}("load1"=>Dict{String, Any}("source_id"=…
  "linecode"       => Dict{String, Any}("zabcn"=>Dict{String, Any}("b_fr"=>[0.0…

In [953]:
eng_3ph_en_2w_yy["transformer"]["tx1"]

KeyError: KeyError: key "tx1" not found

In [954]:
eng_3ph_en_2w_yy["transformer"]["tx1"]["vm_nom"][1] /eng_3ph_en_2w_yy["transformer"]["tx1"]["vm_nom"][2]

KeyError: KeyError: key "tx1" not found

In [955]:
math_3ph_en_2w_yy = transform_data_model(eng_3ph_en_2w_yy, kron_reduce = false, phase_project=false)

Dict{String, Any} with 18 entries:
  "is_kron_reduced" => false
  "conductor_ids"   => [1, 2, 3, 4, 5]
  "time_elapsed"    => 1.0
  "bus"             => Dict{String, Any}("8"=>Dict{String, Any}("vm_pair_lb"=>T…
  "name"            => "test"
  "map"             => Dict{String, Any}[Dict("unmap_function"=>"_map_math2eng_…
  "settings"        => Dict{String, Any}("sbase_default"=>200.0, "vbases_defaul…
  "gen"             => Dict{String, Any}("1"=>Dict{String, Any}("pg"=>[0.0, 0.0…
  "branch"          => Dict{String, Any}("4"=>Dict{String, Any}("br_r"=>[0.0 0.…
  "storage"         => Dict{String, Any}()
  "switch"          => Dict{String, Any}()
  "is_projected"    => false
  "per_unit"        => true
  "data_model"      => MATHEMATICAL
  "shunt"           => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>"s…
  "transformer"     => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>"_…
  "bus_lookup"      => Dict{Any, Int64}("x1"=>1, "x2"=>2, "b2"=>3, "b1"=>4)
  "load"          

In [956]:
transformer_index_byname = findall(x->haskey(x, "from") && x["from"] == "tx1", math_3ph_en_2w_yy["map"])
math_3ph_en_2w_yy["map"][transformer_index_byname...]["to"]


BoundsError: BoundsError: attempt to access 11-element Vector{Dict{String, Any}} at index []

In [957]:
for element in math_3ph_en_2w_yy["map"][transformer_index_byname...]["to"]

    element_type = split(element, ".")[1]
    element_idx = split(element, ".")[2]
    display(element_type)
    display(element_idx)    
    display(math_3ph_en_2w_yy[element_type][element_idx])

end


BoundsError: BoundsError: attempt to access 11-element Vector{Dict{String, Any}} at index []

In [958]:
math_3ph_en_2w_yy["transformer"]

Dict{String, Any} with 2 entries:
  "1" => Dict{String, Any}("source_id"=>"_virtual_transformer.transformer.trans…
  "2" => Dict{String, Any}("source_id"=>"_virtual_transformer.transformer.trans…

In [959]:
math_3ph_en_2w_yy["transformer"]["1"]

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.transformer1.1"
  "t_connections" => [1, 2, 3, 4]
  "f_bus"         => 1
  "polarity"      => 1
  "sm_ub"         => 3.75
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1, 1, 1]
  "tm_lb"         => [0.9, 0.9, 0.9]
  "tm_set"        => [1.02, 1.02, 1.02]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125, 0.03125, 0.03125]
  "t_bus"         => 8
  "f_connections" => [1, 2, 3, 5]
  "configuration" => WYE
  "index"         => 1
  "name"          => "_virtual_transformer.transformer1.1"
  "tm_nom"        => 1.0
  "status"        => 1
  "tm_ub"         => [1.1, 1.1, 1.1]
  "f_vbase"       => 6.35085

In [960]:
math_3ph_en_2w_yy["transformer"]["2"]

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.transformer1.2"
  "t_connections" => [1, 2, 3, 4]
  "f_bus"         => 2
  "polarity"      => 1
  "sm_ub"         => 3.75
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1, 1, 1]
  "tm_lb"         => [0.9, 0.9, 0.9]
  "tm_set"        => [0.97, 0.97, 0.97]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125, 0.03125, 0.03125]
  "t_bus"         => 6
  "f_connections" => [1, 2, 3, 5]
  "configuration" => WYE
  "index"         => 2
  "name"          => "_virtual_transformer.transformer1.2"
  "tm_nom"        => 1.0
  "status"        => 1
  "tm_ub"         => [1.1, 1.1, 1.1]
  "f_vbase"       => 0.23094

In [961]:
(math_3ph_en_2w_yy["transformer"]["1"]["f_vbase"]/math_3ph_en_2w_yy["transformer"]["1"]["t_vbase"]) * (math_3ph_en_2w_yy["transformer"]["2"]["t_vbase"]/math_3ph_en_2w_yy["transformer"]["2"]["f_vbase"])

27.499999999999996

To get the ratio for the whole transformer - you divide the `f_vbase` of the first ideal transformer by the `f_vbase` of the second ideal transformer. (this will match the ratio of the real transformers)

In [962]:
math_3ph_en_2w_yy["transformer"]["1"]["f_vbase"]/math_3ph_en_2w_yy["transformer"]["2"]["f_vbase"]

27.499999999999993

Note this matches whether the LL voltages ration or the LN voltages ratio

In [963]:
# LL voltages ration
11/4

2.75

In [964]:
# LN voltages ration
6.35/2.3094

2.749631939031783

SO Basicallly just the same as the single-phase wye-wye just the difference being that now the loss model (virtual branches) have 3x3 impedance matrices (not 4x4 though!) even if explicit neutral! because there is the packing check. Let's see how this works for delta-wye next!

## Three-Phase Two-Winding Delta-Wye (lead)Transformer

In [965]:
eng_3ph_2w_dy = parse_file(joinpath(dirname(pathof(PowerModelsDistribution)), "..\\test\\data\\opendss\\ut_trans_2w_dy_lead.dss")
, transformations=[transform_loops!] )



[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 1 in 'ut_trans_2w_dy_lead.dss'
[ PowerModelsDistribution | Info ] : Command 'calcvoltagebases' on line 36 in 'ut_trans_2w_dy_lead.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'solve' on line 39 in 'ut_trans_2w_dy_lead.dss' is not supported, skipping.


Dict{String, Any} with 10 entries:
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("source_id"…
  "name"           => "ut_trans"
  "line"           => Dict{String, Any}("line2"=>Dict{String, Any}("cm_ub"=>[60…
  "conductor_ids"  => [1, 2, 3, 4]
  "settings"       => Dict{String, Any}("sbase_default"=>100.0, "vbases_default…
  "files"          => ["C:\\Users\\mnumair\\.julia\\dev\\PowerModelsDistributio…
  "transformer"    => Dict{String, Any}("tx1"=>Dict{String, Any}("polarity"=>[1…
  "load"           => Dict{String, Any}("load3"=>Dict{String, Any}("source_id"=…
  "bus"            => Dict{String, Any}("1"=>Dict{String, Any}("rg"=>[0.0], "gr…
  "data_model"     => ENGINEERING

In [966]:
eng_3ph_2w_dy["transformer"]["tx1"]

Dict{String, Any} with 19 entries:
  "polarity"      => [1, 1]
  "sm_nom"        => [500.0, 500.0]
  "tm_lb"         => [[0.9, 0.9, 0.9], [0.9, 0.9, 0.9]]
  "connections"   => [[1, 2, 3], [1, 2, 3, 4]]
  "tm_set"        => [[1.02, 1.02, 1.02], [0.97, 0.97, 0.97]]
  "tm_step"       => [[0.03125, 0.03125, 0.03125], [0.03125, 0.03125, 0.03125]]
  "bus"           => ["1", "2"]
  "configuration" => ConnConfig[DELTA, WYE]
  "name"          => "tx1"
  "status"        => ENABLED
  "noloadloss"    => 0.05
  "cmag"          => 0.11
  "xsc"           => [0.05]
  "source_id"     => "transformer.tx1"
  "sm_ub"         => 750.0
  "rw"            => [0.01, 0.02]
  "tm_fix"        => Vector{Bool}[[1, 1, 1], [1, 1, 1]]
  "vm_nom"        => [11.0, 4.0]
  "tm_ub"         => [[1.1, 1.1, 1.1], [1.1, 1.1, 1.1]]

In [967]:
eng_3ph_2w_dy["transformer"]["tx1"]["vm_nom"][1] /eng_3ph_2w_dy["transformer"]["tx1"]["vm_nom"][2]

2.75

In [968]:
math_3ph_2w_dy = transform_data_model(eng_3ph_2w_dy, kron_reduce = false, phase_project=false)

Dict{String, Any} with 18 entries:
  "is_kron_reduced" => false
  "conductor_ids"   => [1, 2, 3, 4]
  "time_elapsed"    => 1.0
  "bus"             => Dict{String, Any}("8"=>Dict{String, Any}("vm_pair_lb"=>T…
  "name"            => "ut_trans"
  "map"             => Dict{String, Any}[Dict("unmap_function"=>"_map_math2eng_…
  "settings"        => Dict{String, Any}("sbase_default"=>100.0, "vbases_defaul…
  "gen"             => Dict{String, Any}("1"=>Dict{String, Any}("pg"=>[0.0, 0.0…
  "branch"          => Dict{String, Any}("4"=>Dict{String, Any}("br_r"=>[0.0 0.…
  "storage"         => Dict{String, Any}()
  "switch"          => Dict{String, Any}()
  "is_projected"    => false
  "per_unit"        => true
  "data_model"      => MATHEMATICAL
  "shunt"           => Dict{String, Any}()
  "transformer"     => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>"_…
  "bus_lookup"      => Dict{Any, Int64}("1"=>1, "sourcebus"=>2, "2"=>3, "3"=>4)
  "load"            => Dict{String, Any}("4"=>Dict{

In [969]:
transformer_index_byname = findall(x->haskey(x, "from") && x["from"] == "tx1", math_3ph_2w_dy["map"])
math_3ph_2w_dy["map"][transformer_index_byname...]["to"]


9-element Vector{String}:
 "bus.5"
 "bus.6"
 "bus.7"
 "bus.8"
 "branch.3"
 "branch.4"
 "branch.5"
 "transformer.1"
 "transformer.2"

In [970]:
for element in math_3ph_2w_dy["map"][transformer_index_byname...]["to"]

    element_type = split(element, ".")[1]
    element_idx = split(element, ".")[2]
    display(element_type)
    display(element_idx)    
    display(math_3ph_2w_dy[element_type][element_idx])

end


"bus"

"5"

Dict{String, Any} with 13 entries:
  "vm_pair_lb" => Tuple{Any, Any, Real}[]
  "source_id"  => "transformer.tx1"
  "grounded"   => Bool[0, 0, 0]
  "vmin"       => [0.0, 0.0, 0.0]
  "vm_pair_ub" => Tuple{Any, Any, Real}[]
  "bus_i"      => 5
  "index"      => 5
  "name"       => "_virtual_bus.transformer.tx1_4"
  "bus_type"   => 1
  "terminals"  => [1, 2, 3]
  "vmax"       => [Inf, Inf, Inf]
  "base_kv"    => 1.0
  "vbase"      => 0.57735

"bus"

"6"

...

"1"

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.tx1.1"
  "t_connections" => [1, 2, 3, 4]
  "f_bus"         => 1
  "polarity"      => 1
  "sm_ub"         => 7.5
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1, 1, 1]
  "tm_lb"         => [0.9, 0.9, 0.9]
  "tm_set"        => [1.02, 1.02, 1.02]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125, 0.03125, 0.03125]
  "t_bus"         => 8
  "f_connections" => [1, 2, 3]
  "configuration" => DELTA
  "index"         => 1
  "name"          => "_virtual_transformer.tx1.1"
  "tm_nom"        => 1.73205
  "status"        => 1
  "tm_ub"         => [1.1, 1.1, 1.1]
  "f_vbase"       => 6.35085

"transformer"

"2"

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.tx1.2"
  "t_connections" => [1, 2, 3, 4]
  "f_bus"         => 3
  "polarity"      => 1
  "sm_ub"         => 7.5
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1, 1, 1]
  "tm_lb"         => [0.9, 0.9, 0.9]
  "tm_set"        => [0.97, 0.97, 0.97]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125, 0.03125, 0.03125]
  "t_bus"         => 6
  "f_connections" => [1, 2, 3, 4]
  "configuration" => WYE
  "index"         => 2
  "name"          => "_virtual_transformer.tx1.2"
  "tm_nom"        => 1.0
  "status"        => 1
  "tm_ub"         => [1.1, 1.1, 1.1]
  "f_vbase"       => 2.3094

In [971]:
math_3ph_2w_dy["transformer"]

Dict{String, Any} with 2 entries:
  "1" => Dict{String, Any}("source_id"=>"_virtual_transformer.transformer.tx1.1…
  "2" => Dict{String, Any}("source_id"=>"_virtual_transformer.transformer.tx1.2…

In [972]:
math_3ph_2w_dy["transformer"]["1"]

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.tx1.1"
  "t_connections" => [1, 2, 3, 4]
  "f_bus"         => 1
  "polarity"      => 1
  "sm_ub"         => 7.5
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1, 1, 1]
  "tm_lb"         => [0.9, 0.9, 0.9]
  "tm_set"        => [1.02, 1.02, 1.02]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125, 0.03125, 0.03125]
  "t_bus"         => 8
  "f_connections" => [1, 2, 3]
  "configuration" => DELTA
  "index"         => 1
  "name"          => "_virtual_transformer.tx1.1"
  "tm_nom"        => 1.73205
  "status"        => 1
  "tm_ub"         => [1.1, 1.1, 1.1]
  "f_vbase"       => 6.35085

In [973]:
math_3ph_2w_dy["transformer"]["2"]

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.tx1.2"
  "t_connections" => [1, 2, 3, 4]
  "f_bus"         => 3
  "polarity"      => 1
  "sm_ub"         => 7.5
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1, 1, 1]
  "tm_lb"         => [0.9, 0.9, 0.9]
  "tm_set"        => [0.97, 0.97, 0.97]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125, 0.03125, 0.03125]
  "t_bus"         => 6
  "f_connections" => [1, 2, 3, 4]
  "configuration" => WYE
  "index"         => 2
  "name"          => "_virtual_transformer.tx1.2"
  "tm_nom"        => 1.0
  "status"        => 1
  "tm_ub"         => [1.1, 1.1, 1.1]
  "f_vbase"       => 2.3094

In [974]:
(math_3ph_2w_dy["transformer"]["1"]["f_vbase"]/math_3ph_2w_dy["transformer"]["1"]["t_vbase"]) * (math_3ph_2w_dy["transformer"]["2"]["t_vbase"]/math_3ph_2w_dy["transformer"]["2"]["f_vbase"])

2.7499999999999996

To get the ratio for the whole transformer - you divide the `f_vbase` of the first ideal transformer by the `f_vbase` of the second ideal transformer. (this will match the ratio of the real transformers)

In [975]:
math_3ph_2w_dy["transformer"]["1"]["f_vbase"]/math_3ph_2w_dy["transformer"]["2"]["f_vbase"]

2.7499999999999996

Note this matches whether the LL voltages ration or the LN voltages ratio

In [976]:
# LL voltages ration
11/0.4

27.5

In [977]:
# LN voltages ration
6.35/0.23094

27.49631939031783

SO Basicallly just the same as the single-phase wye-wye just the difference being that now the loss model (virtual branches) have 3x3 impedance matrices (not 4x4 though!) even if explicit neutral! because there is the packing check. Let's see how this works for delta-wye next!

## Three-Phase Explicit Neutral Two-Winding Delta-Wye (lag) Transformer   

In [978]:
eng_3ph_en_2w_dy = parse_file(joinpath(dirname(pathof(PowerModelsDistribution)), "..\\test\\data\\en_validation_case_data\\test_trans_dy.dss"), transformations=[transform_loops!] )


[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 3 in 'test_trans_dy.dss'
[ PowerModelsDistribution | Info ] : Command 'calcvoltagebases' on line 36 in 'test_trans_dy.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'solve' on line 38 in 'test_trans_dy.dss' is not supported, skipping.
┌ PowerModelsDistribution | Warning ] : reactors as constant impedance elements is not yet supported, treating reactor.grounding like line
└ @ PowerModelsDistribution C:\Users\mnumair\.julia\dev\PowerModelsDistribution\src\data_model\transformations\dss2eng.jl:263


Dict{String, Any} with 12 entries:
  "conductor_ids"  => [1, 2, 3, 4, 5]
  "bus"            => Dict{String, Any}("x1"=>Dict{String, Any}("rg"=>Float64[]…
  "name"           => "test"
  "settings"       => Dict{String, Any}("sbase_default"=>200.0, "vbases_default…
  "files"          => ["C:\\Users\\mnumair\\.julia\\dev\\PowerModelsDistributio…
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("source_id"…
  "line"           => Dict{String, Any}("line2"=>Dict{String, Any}("length"=>50…
  "data_model"     => ENGINEERING
  "transformer"    => Dict{String, Any}("transformer1"=>Dict{String, Any}("pola…
  "shunt"          => Dict{String, Any}("line_loop.grounding"=>Dict{String, Any…
  "load"           => Dict{String, Any}("load1"=>Dict{String, Any}("source_id"=…
  "linecode"       => Dict{String, Any}("zabcn"=>Dict{String, Any}("b_fr"=>[0.0…

In [979]:
eng_3ph_en_2w_dy["transformer"]["tx1"]

KeyError: KeyError: key "tx1" not found

In [980]:
eng_3ph_en_2w_dy["transformer"]["tx1"]["vm_nom"][1] /eng_3ph_en_2w_dy["transformer"]["tx1"]["vm_nom"][2]

KeyError: KeyError: key "tx1" not found

In [981]:
math_3ph_en_2w_dy = transform_data_model(eng_3ph_en_2w_dy, kron_reduce = false, phase_project=false)

Dict{String, Any} with 18 entries:
  "is_kron_reduced" => false
  "conductor_ids"   => [1, 2, 3, 4, 5]
  "time_elapsed"    => 1.0
  "bus"             => Dict{String, Any}("8"=>Dict{String, Any}("vm_pair_lb"=>T…
  "name"            => "test"
  "map"             => Dict{String, Any}[Dict("unmap_function"=>"_map_math2eng_…
  "settings"        => Dict{String, Any}("sbase_default"=>200.0, "vbases_defaul…
  "gen"             => Dict{String, Any}("1"=>Dict{String, Any}("pg"=>[0.0, 0.0…
  "branch"          => Dict{String, Any}("4"=>Dict{String, Any}("br_r"=>[0.0 0.…
  "storage"         => Dict{String, Any}()
  "switch"          => Dict{String, Any}()
  "is_projected"    => false
  "per_unit"        => true
  "data_model"      => MATHEMATICAL
  "shunt"           => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>"s…
  "transformer"     => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>"_…
  "bus_lookup"      => Dict{Any, Int64}("x1"=>1, "x2"=>2, "b2"=>3, "b1"=>4)
  "load"          

In [982]:
transformer_index_byname = findall(x->haskey(x, "from") && x["from"] == "tx1", math_3ph_en_2w_dy["map"])
math_3ph_en_2w_dy["map"][transformer_index_byname...]["to"]


BoundsError: BoundsError: attempt to access 11-element Vector{Dict{String, Any}} at index []

In [983]:
for element in math_3ph_en_2w_dy["map"][transformer_index_byname...]["to"]

    element_type = split(element, ".")[1]
    element_idx = split(element, ".")[2]
    display(element_type)
    display(element_idx)    
    display(math_3ph_en_2w_dy[element_type][element_idx])

end


BoundsError: BoundsError: attempt to access 11-element Vector{Dict{String, Any}} at index []

In [984]:
math_3ph_en_2w_dy["transformer"]

Dict{String, Any} with 2 entries:
  "1" => Dict{String, Any}("source_id"=>"_virtual_transformer.transformer.trans…
  "2" => Dict{String, Any}("source_id"=>"_virtual_transformer.transformer.trans…

In [985]:
math_3ph_en_2w_dy["transformer"]["1"]

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.transformer1.1"
  "t_connections" => [1, 2, 3, 4]
  "f_bus"         => 1
  "polarity"      => 1
  "sm_ub"         => 3.75
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1, 1, 1]
  "tm_lb"         => [0.9, 0.9, 0.9]
  "tm_set"        => [1.02, 1.02, 1.02]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125, 0.03125, 0.03125]
  "t_bus"         => 8
  "f_connections" => [1, 2, 3]
  "configuration" => DELTA
  "index"         => 1
  "name"          => "_virtual_transformer.transformer1.1"
  "tm_nom"        => 1.73205
  "status"        => 1
  "tm_ub"         => [1.1, 1.1, 1.1]
  "f_vbase"       => 6.35085

In [986]:
math_3ph_en_2w_dy["transformer"]["2"]

Dict{String, Any} with 20 entries:
  "source_id"     => "_virtual_transformer.transformer.transformer1.2"
  "t_connections" => [1, 2, 3, 4]
  "f_bus"         => 2
  "polarity"      => -1
  "sm_ub"         => 3.75
  "cm_ub"         => Inf
  "tm_fix"        => Bool[1, 1, 1]
  "tm_lb"         => [0.9, 0.9, 0.9]
  "tm_set"        => [0.97, 0.97, 0.97]
  "t_vbase"       => 0.57735
  "tm_step"       => [0.03125, 0.03125, 0.03125]
  "t_bus"         => 6
  "f_connections" => [1, 3, 2, 5]
  "configuration" => WYE
  "index"         => 2
  "name"          => "_virtual_transformer.transformer1.2"
  "tm_nom"        => 1.0
  "status"        => 1
  "tm_ub"         => [1.1, 1.1, 1.1]
  "f_vbase"       => 0.23094

In [987]:
(math_3ph_en_2w_dy["transformer"]["1"]["f_vbase"]/math_3ph_en_2w_dy["transformer"]["1"]["t_vbase"]) * (math_3ph_en_2w_dy["transformer"]["2"]["t_vbase"]/math_3ph_en_2w_dy["transformer"]["2"]["f_vbase"])

27.499999999999996

To get the ratio for the whole transformer - you divide the `f_vbase` of the first ideal transformer by the `f_vbase` of the second ideal transformer. (this will match the ratio of the real transformers)

In [988]:
math_3ph_en_2w_dy["transformer"]["1"]["f_vbase"]/math_3ph_en_2w_dy["transformer"]["2"]["f_vbase"]

27.499999999999993

Note this matches whether the LL voltages ration or the LN voltages ratio

In [989]:
# LL voltages ration
11/0.4

27.5

In [990]:
# LN voltages ration
6.35/0.23094

27.49631939031783

SO Basicallly just the same as the single-phase wye-wye just the difference being that now the loss model (virtual branches) have 3x3 impedance matrices (not 4x4 though!) even if explicit neutral! because there is the packing check. Let's see how this works for delta-wye next!

In [991]:
# Pkg.add("Ipopt")
# using Ipopt

In [992]:
using Ipopt

In [993]:
pf_1ph_yy = compute_mc_pf(math_1ph_2w_yy)

Dict{String, Any} with 8 entries:
  "iterations"         => 3
  "time_total"         => 0.0013956
  "time_build"         => 0.0009962
  "termination_status" => PF_CONVERGED
  "solution"           => Dict{String, Any}("gen"=>Dict{String, Any}("1"=>Dict{…
  "time_post"          => 0.0002133
  "stationarity"       => 4.97493e-11
  "time_solve"         => 0.0001861

In [994]:
pf_1ph_yy["solution"]["bus"]

Dict{String, Any} with 9 entries:
  "3" => Dict{String, Any}("va"=>[0.00066113], "vm"=>[0.897988])
  "4" => Dict{String, Any}("va"=>[0.000730898], "vm"=>[0.8978])
  "1" => Dict{String, Any}("va"=>[-5.57917e-11, -2.0944, 2.0944], "vm"=>[1.0, 1…
  "5" => Dict{String, Any}("va"=>[-0.00167054], "vm"=>[0.946326])
  "2" => Dict{String, Any}("va"=>[4.88257e-7], "vm"=>[0.999994])
  "6" => Dict{String, Any}("va"=>[0.00066113], "vm"=>[0.945251])
  "7" => Dict{String, Any}("va"=>[0.0011592], "vm"=>[0.951839])
  "9" => Dict{String, Any}("va"=>[0.0, -2.0944, 2.0944], "vm"=>[1.0, 1.0, 1.0])
  "8" => Dict{String, Any}("va"=>[4.88257e-7], "vm"=>[0.952376])

In [995]:
compute_mc_pf(math_3ph_2w_yy, max_iter=50000)

Dict{String, Any} with 8 entries:
  "iterations"         => 13
  "time_total"         => 0.0016198
  "time_build"         => 0.0010237
  "termination_status" => PF_CONVERGED
  "solution"           => Dict{String, Any}("gen"=>Dict{String, Any}("1"=>Dict{…
  "time_post"          => 0.0002279
  "stationarity"       => 2.08721e-9
  "time_solve"         => 0.0003682

In [996]:
add_start_vrvi!(math_3ph_en_2w_yy)

Dict{String, Any} with 18 entries:
  "is_kron_reduced" => false
  "conductor_ids"   => [1, 2, 3, 4, 5]
  "time_elapsed"    => 1.0
  "bus"             => Dict{String, Any}("8"=>Dict{String, Any}("vm_pair_lb"=>T…
  "name"            => "test"
  "map"             => Dict{String, Any}[Dict("unmap_function"=>"_map_math2eng_…
  "settings"        => Dict{String, Any}("sbase_default"=>200.0, "vbases_defaul…
  "gen"             => Dict{String, Any}("1"=>Dict{String, Any}("pg"=>[0.0, 0.0…
  "branch"          => Dict{String, Any}("4"=>Dict{String, Any}("br_r"=>[0.0 0.…
  "storage"         => Dict{String, Any}()
  "switch"          => Dict{String, Any}()
  "is_projected"    => false
  "per_unit"        => true
  "data_model"      => MATHEMATICAL
  "shunt"           => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>"s…
  "transformer"     => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>"_…
  "bus_lookup"      => Dict{Any, Int64}("x1"=>1, "x2"=>2, "b2"=>3, "b1"=>4)
  "load"          

In [997]:
zones = PowerModelsDistribution._discover_voltage_zones(math_3ph_en_2w_yy, PowerModelsDistribution._math_edge_elements)
bus_to_zone = Dict([(bus, zone) for (zone, buses) in zones for bus in buses])

vbases = Dict(i => v * math_3ph_en_2w_yy["settings"]["voltage_scale_factor"] for (i, v) in math_3ph_en_2w_yy["settings"]["vbases_default"])
sbase = math_3ph_en_2w_yy["settings"]["sbase_default"] * math_3ph_en_2w_yy["settings"]["power_scale_factor"]
bus_vbase, line_vbase = calc_voltage_bases(math_3ph_en_2w_yy, vbases)

line_vbase
bus_vbase



Dict{String, Float64} with 9 entries:
  "3" => 6350.85
  "4" => 6350.85
  "1" => 6350.85
  "5" => 6350.85
  "2" => 6350.85
  "7" => 6350.85
  "6" => 6350.85
  "9" => 6350.85
  "8" => 6350.85

In [998]:
math_3ph_2w_yy

Dict{String, Any} with 18 entries:
  "is_kron_reduced" => true
  "conductor_ids"   => [1, 2, 3]
  "time_elapsed"    => 1.0
  "bus"             => Dict{String, Any}("8"=>Dict{String, Any}("vm_pair_lb"=>T…
  "name"            => "ut_trans"
  "map"             => Dict{String, Any}[Dict("unmap_function"=>"_map_math2eng_…
  "settings"        => Dict{String, Any}("sbase_default"=>1.0, "vbases_default"…
  "gen"             => Dict{String, Any}("1"=>Dict{String, Any}("pg"=>[0.0, 0.0…
  "branch"          => Dict{String, Any}("4"=>Dict{String, Any}("br_r"=>[0.0 0.…
  "storage"         => Dict{String, Any}()
  "switch"          => Dict{String, Any}()
  "is_projected"    => true
  "per_unit"        => true
  "data_model"      => MATHEMATICAL
  "shunt"           => Dict{String, Any}()
  "transformer"     => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>"_…
  "bus_lookup"      => Dict{Any, Int64}("1"=>1, "sourcebus"=>2, "2"=>3, "3"=>4)
  "load"            => Dict{String, Any}("4"=>Dict{Strin

In [1001]:
solve_mc_opf(math_3ph_2w_yy, ACPUPowerModel, Ipopt.Optimizer)

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.8.1.

Number of nonzeros in equality constraint Jacobian...:      909
Number of nonzeros in inequality constraint Jacobian.:      108
Number of nonzeros in Lagrangian Hessian.............:     1872

Total number of variables............................:      153
                     variables with only lower bounds:       24
                variables with lower and upper bounds:       24
                     variables with only upper bounds:        0
Total number of equality constraints.................:      153
Total number of inequality constraints...............:       48
        inequality constraints with only lower bounds:       18
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       30

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 9.60e+01 1.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

Dict{String, Any} with 8 entries:
  "solve_time"         => 0.0369999
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 0.467575
  "solution"           => Dict{String, Any}("branch"=>Dict{String, Any}("4"=>Di…
  "objective_lb"       => -Inf

In [1002]:
compute_mc_pf(math_3ph_2w_yy)

Dict{String, Any} with 8 entries:
  "iterations"         => 13
  "time_total"         => 0.0014535
  "time_build"         => 0.0008332
  "termination_status" => PF_CONVERGED
  "solution"           => Dict{String, Any}("gen"=>Dict{String, Any}("1"=>Dict{…
  "time_post"          => 0.0002126
  "stationarity"       => 2.08721e-9
  "time_solve"         => 0.0004077

In [1003]:
pf_yy_en = compute_mc_pf(math_3ph_en_2w_yy, explicit_neutral=true)

Dict{String, Any} with 8 entries:
  "iterations"         => 10
  "time_total"         => 0.0014104
  "time_build"         => 0.0009315
  "termination_status" => PF_CONVERGED
  "solution"           => Dict{String, Any}("gen"=>Dict{String, Any}("1"=>Dict{…
  "time_post"          => 0.0002007
  "stationarity"       => 9.12045e-9
  "time_solve"         => 0.0002782

In [1004]:
math_3ph_en_2w_yy["bus"]["4"]

Dict{String, Any} with 14 entries:
  "vm_pair_lb" => Tuple{Any, Any, Real}[]
  "grounded"   => Bool[0, 0, 0, 1]
  "vm_pair_ub" => Tuple{Any, Any, Real}[]
  "bus_i"      => 4
  "vr_start"   => [1.0, -0.5, -0.5, 0.0]
  "name"       => "b1"
  "bus_type"   => 1
  "terminals"  => [1, 2, 3, 4]
  "vmax"       => [Inf, Inf, Inf, Inf]
  "vbase"      => 6.35085
  "source_id"  => "bus.b1"
  "vmin"       => [0.0, 0.0, 0.0, 0.0]
  "index"      => 4
  "vi_start"   => [0.0, -0.866025, 0.866025, 0.0]

In [1005]:
vbase = math_3ph_en_2w_yy["settings"]["vbases_default"]["2"]*math_3ph_en_2w_yy["settings"]["voltage_scale_factor"]

KeyError: KeyError: key "2" not found

In [1006]:
pf_yy_en["solution"]["bus"]["4"]["vm"]

4-element Vector{Float64}:
 0.9999999991917288
 0.9999999994312019
 0.9999999994454145
 0.0

In [1007]:
pf_yy_en["solution"]["bus"]["4"]["vm"].*(0.4/sqrt(3))

4-element Vector{Float64}:
 0.23094010748918808
 0.23094010754449204
 0.2309401075477743
 0.0

In [1008]:
rad2deg.(pf_yy_en["solution"]["bus"]["4"]["va"])

4-element Vector{Float64}:
   -1.8926432372895567e-8
 -120.00000000642756
  119.99999999407059
    0.0

In [1009]:
solve_mc_opf(math_3ph_2w_yy, ACPUPowerModel, Ipopt.Optimizer)

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.8.1.

Number of nonzeros in equality constraint Jacobian...:      909
Number of nonzeros in inequality constraint Jacobian.:      108
Number of nonzeros in Lagrangian Hessian.............:     1872

Total number of variables............................:      153
                     variables with only lower bounds:       24
                variables with lower and upper bounds:       24
                     variables with only upper bounds:        0
Total number of equality constraints.................:      153
Total number of inequality constraints...............:       48
        inequality constraints with only lower bounds:       18
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       30

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 9.60e+01 1.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

Dict{String, Any} with 8 entries:
  "solve_time"         => 0.0309999
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 0.467575
  "solution"           => Dict{String, Any}("branch"=>Dict{String, Any}("4"=>Di…
  "objective_lb"       => -Inf

In [1010]:
solve_mc_opf(math_3ph_2w_dy, ACPUPowerModel, Ipopt.Optimizer)


BoundsError: BoundsError: attempt to access 3-element Vector{Float64} at index [4]

In [1011]:
pf_dy = compute_mc_pf(math_3ph_2w_dy, explicit_neutral=false)

DimensionMismatch: DimensionMismatch: arrays could not be broadcast to a common size: a has axes Base.OneTo(3) and b has axes Base.OneTo(4)

In [1012]:
solve_mc_opf(math_3ph_2w_dy, IVRENPowerModel, Ipopt.Optimizer)


This is Ipopt version 3.14.19, running with linear solver MUMPS 5.8.1.

Number of nonzeros in equality constraint Jacobian...:      654
Number of nonzeros in inequality constraint Jacobian.:       84
Number of nonzeros in Lagrangian Hessian.............:      402

The Jacobian for the equality constraints contains an invalid number

Number of Iterations....: 0

Number of objective function evaluations             = 0
Number of objective gradient evaluations             = 0
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations          = 1
Number of equality constraint Jacobian evaluations   = 1
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 0
Total seconds in IPOPT                               = 0.000

EXIT: Invalid number in NLP function or derivative detected.


Dict{String, Any} with 8 entries:
  "solve_time"         => 0.00200009
  "optimizer"          => "Ipopt"
  "termination_status" => INVALID_MODEL
  "dual_status"        => UNKNOWN_RESULT_STATUS
  "primal_status"      => INFEASIBLE_POINT
  "objective"          => 0.0
  "solution"           => Dict{String, Any}("branch"=>Dict{String, Any}("4"=>Di…
  "objective_lb"       => -Inf

In [1013]:
eng_test_trans_yy = parse_file(path_test_trans_yy, transformations=[transform_loops!] )

[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 3 in 'test_trans_yy.dss'
[ PowerModelsDistribution | Info ] : Command 'calcvoltagebases' on line 36 in 'test_trans_yy.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'solve' on line 38 in 'test_trans_yy.dss' is not supported, skipping.
┌ PowerModelsDistribution | Warning ] : reactors as constant impedance elements is not yet supported, treating reactor.grounding like line
└ @ PowerModelsDistribution C:\Users\mnumair\.julia\dev\PowerModelsDistribution\src\data_model\transformations\dss2eng.jl:263


Dict{String, Any} with 12 entries:
  "conductor_ids"  => [1, 2, 3, 4, 5]
  "bus"            => Dict{String, Any}("x1"=>Dict{String, Any}("rg"=>[0.0], "g…
  "name"           => "test"
  "settings"       => Dict{String, Any}("sbase_default"=>200.0, "vbases_default…
  "files"          => ["C:\\Users\\mnumair\\.julia\\dev\\PowerModelsDistributio…
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("source_id"…
  "line"           => Dict{String, Any}("line2"=>Dict{String, Any}("length"=>50…
  "data_model"     => ENGINEERING
  "transformer"    => Dict{String, Any}("transformer1"=>Dict{String, Any}("pola…
  "shunt"          => Dict{String, Any}("line_loop.grounding"=>Dict{String, Any…
  "load"           => Dict{String, Any}("load1"=>Dict{String, Any}("source_id"=…
  "linecode"       => Dict{String, Any}("zabcn"=>Dict{String, Any}("b_fr"=>[0.0…

In [1014]:
math_test_trans_yy = transform_data_model(eng_test_trans_yy, kron_reduce = false, phase_project=false)

Dict{String, Any} with 18 entries:
  "is_kron_reduced" => false
  "conductor_ids"   => [1, 2, 3, 4, 5]
  "time_elapsed"    => 1.0
  "bus"             => Dict{String, Any}("8"=>Dict{String, Any}("vm_pair_lb"=>T…
  "name"            => "test"
  "map"             => Dict{String, Any}[Dict("unmap_function"=>"_map_math2eng_…
  "settings"        => Dict{String, Any}("sbase_default"=>200.0, "vbases_defaul…
  "gen"             => Dict{String, Any}("1"=>Dict{String, Any}("pg"=>[0.0, 0.0…
  "branch"          => Dict{String, Any}("4"=>Dict{String, Any}("br_r"=>[0.0 0.…
  "storage"         => Dict{String, Any}()
  "switch"          => Dict{String, Any}()
  "is_projected"    => false
  "per_unit"        => true
  "data_model"      => MATHEMATICAL
  "shunt"           => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>"s…
  "transformer"     => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>"_…
  "bus_lookup"      => Dict{Any, Int64}("x1"=>1, "x2"=>2, "b2"=>3, "b1"=>4)
  "load"          